**trainer**

In [4]:
import cv2
import os
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
from threading import Thread, Lock
import time
import numpy as np
import dlib
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

In [5]:
#RTSP_URL = "rtsp://wellantmilieu@gmail.com:q8oEZQC!W$8d@192.168.178.22:554/stream2"
#cap = cv2.VideoCapture(1) #of 0

In [6]:
BASE_DIR = "datasets"
os.makedirs(BASE_DIR, exist_ok=True)

### model trainer met LBPH (Local Binary Pattern Histogram)

In [7]:
# Ophalen van beschikbare dataset-mappen
available_datasets = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]

# Dropdown-menu voor dataset-selectie
dataset_dropdown = widgets.Dropdown(
    options=available_datasets,
    description="Selecteer Naam:"
)
train_button = widgets.Button(description="Train Model")
output = widgets.Output()

# Weergave in Notebook
display(dataset_dropdown, train_button, output)

# **📌 Functie om een afbeelding te verbeteren**
def preprocess_image(img):
    """ Zorgt voor betere herkenning door bewerking van afbeeldingen """
    img = cv2.equalizeHist(img)  # ✅ Helderheid normaliseren
    img = cv2.GaussianBlur(img, (3, 3), 0)  # ✅ Ruis verminderen
    return img

# **📌 Data augmentatie toepassen**
def augment_image(img):
    """ Voegt variatie toe aan trainingsdata """
    augmented_images = []

    # Oorspronkelijke afbeelding
    augmented_images.append(img)

    # Horizontale spiegeling
    augmented_images.append(cv2.flip(img, 1))

    # Kleine rotaties
    for angle in [-10, 10]:  
        rows, cols = img.shape
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        rotated = cv2.warpAffine(img, M, (cols, rows))
        augmented_images.append(rotated)

    # Lichte contrastwijzigingen
    alpha = random.uniform(0.8, 1.2)  # Contrast (0.8-1.2)
    beta = random.randint(-20, 20)  # Helderheid (-20 tot 20)
    contrast = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    augmented_images.append(contrast)

    return augmented_images

# **📌 Train het model met betere data**
def train_model(_):
    selected_name = dataset_dropdown.value
    if not selected_name:
        with output:
            print("Selecteer een dataset om te trainen.")
        return

    PROCESSED_DIR = os.path.join(BASE_DIR, selected_name, "processed_faces")
    MODEL_PATH = os.path.join(BASE_DIR, selected_name, f"face_model_{selected_name}.yml")

    # Controleren of er gezichten beschikbaar zijn
    image_files = [f for f in os.listdir(PROCESSED_DIR) if f.endswith(".jpg")]
    if not image_files:
        with output:
            print(f"Geen gezichten gevonden in {PROCESSED_DIR}, training geannuleerd.")
        return

    # Gegevens en labels verzamelen
    images = []
    labels = []
    label_dict = {name: idx for idx, name in enumerate(available_datasets)}  # ✅ Unieke labels per persoon

    for filename in image_files:
        img_path = os.path.join(PROCESSED_DIR, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            with output:
                print(f"⚠️ Kan afbeelding {filename} niet laden, overslaan.")
            continue

        # **✅ Verbeter de afbeelding en pas data augmentation toe**
        img = preprocess_image(img)
        augmented_imgs = augment_image(img)

        for aug_img in augmented_imgs:
            aug_img_resized = cv2.resize(aug_img, (200, 200))
            images.append(aug_img_resized)
            labels.append(label_dict[selected_name])

    # **🚀 Check de datasetgrootte**
    print(f"📊 Totaal {len(images)} beelden na augmentatie.")

    # **📌 Model trainen**
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(np.array(images, dtype=np.uint8), np.array(labels, dtype=np.int32))

    # **📂 Opslaan in `datasets/<naam>/`**
    model.save(MODEL_PATH)

    with output:
        print(f"✅ Model getraind en opgeslagen als {MODEL_PATH} met {len(images)} beelden.")

# Koppel knop aan functie
train_button.on_click(train_model)


Dropdown(description='Selecteer Naam:', options=('Arend', 'Bobby', 'Joost', 'Marieke_2', 'PJ', 'Robert', 'Unkn…

Button(description='Train Model', style=ButtonStyle())

Output()

### modeltrainer met CCN

In [4]:
MODEL_PATH = "face_recognition_modelv2.h5"
LABEL_PATH = "label_mapping.npy"

# Ophalen van beschikbare dataset-mappen
available_datasets = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d, "processed_faces"))]

# Train-knop en output venster
train_button = widgets.Button(description="Train Model")
output = widgets.Output()
display(train_button, output)

def train_model(_):
    images = []
    labels = []
    label_dict = {}

    # Verzamel ALLE gezichten per persoon (processed_faces)
    person_images = {}
    
    for idx, person in enumerate(available_datasets):
        person_path = os.path.join(BASE_DIR, person, "processed_faces")
        image_files = [os.path.join(person_path, f) for f in os.listdir(person_path) if f.endswith(".jpg")]

        if len(image_files) < 5:  # Minimaal 5 afbeeldingen vereist
            with output:
                print(f"❌ Onvoldoende foto's voor {person}, overslaan.")
            continue
        
        label_dict[person] = idx  # Koppel naam aan label index
        person_images[idx] = image_files

    if not person_images:
        with output:
            print("❌ Geen geldige afbeeldingen gevonden voor training.")
        return

    # **Zorg dat alle labels evenveel afbeeldingen hebben**
    min_images = min(len(imgs) for imgs in person_images.values())
    balanced_images = []
    balanced_labels = []
    
    for label, image_files in person_images.items():
        selected_files = np.random.choice(image_files, min_images, replace=False)  # Random selecteren
        for img_path in selected_files:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_resized = cv2.resize(img, (100, 100))
            balanced_images.append(img_resized)
            balanced_labels.append(label)

    # **Converteer naar numpy arrays en normaliseer**
    images = np.array(balanced_images, dtype="float32") / 255.0
    labels = np.array(balanced_labels, dtype="int32")

    # **Gebruik Data Augmentation**
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    augmented_images = []
    augmented_labels = []

    for img, label in zip(images, labels):
        img = np.expand_dims(img, axis=0)  # Expand dims voor ImageDataGenerator
        augment_iter = datagen.flow(img, batch_size=1)
        
        for _ in range(3):  # Voeg 3 augmented versies toe per afbeelding
            augmented_img = next(augment_iter)[0]
            augmented_images.append(augmented_img)
            augmented_labels.append(label)

    # Voeg augmented data toe aan dataset
    images = np.concatenate([images, np.array(augmented_images)], axis=0)
    labels = np.concatenate([labels, np.array(augmented_labels)], axis=0)

    # **Shuffle de dataset om bias te voorkomen**
    images, labels = shuffle(images, labels, random_state=42)

    # **Verbeterd model met Dropout**
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Voorkomt overfitting
        Dense(len(label_dict), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    model.fit(images, labels, epochs=15, batch_size=32, validation_split=0.2)

    model.save(MODEL_PATH)
    np.save(LABEL_PATH, label_dict)

    with output:
        print(f"✅ Model opgeslagen als {MODEL_PATH} en labels als {LABEL_PATH}")
        unique, counts = np.unique(labels, return_counts=True)
        print("Aantal afbeeldingen per label:", dict(zip(unique, counts)))
        print("Label mapping:", label_dict)

train_button.on_click(train_model)


Button(description='Train Model', style=ButtonStyle())

Output()

### update

In [ ]:
# Paden voor datasets, model en labels
BASE_DIR = "datasets"
MODEL_PATH = "face_recognition_model_git.h5"
LABEL_PATH = "label_mapping_git.npy"

# Ophalen van beschikbare dataset-mappen die een "processed_faces" submap hebben
available_datasets = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d, "processed_faces"))]

# Widgets: train-knop en output-venster
train_button = widgets.Button(description="Train Model")
output = widgets.Output()
display(train_button, output)

def train_model(_):
    with output:
        output.clear_output()
        print("Start training...")
        
    images = []
    labels = []
    label_dict = {}  # mapping: label_index -> persoon
    person_images = {}

    # Bouw de labelmapping en verzamel paden naar afbeeldingen per persoon
    for idx, person in enumerate(available_datasets):
        person_path = os.path.join(BASE_DIR, person, "processed_faces")
        image_files = [os.path.join(person_path, f) for f in os.listdir(person_path) if f.endswith(".jpg")]
        
        if len(image_files) < 5:  # minimaal 5 afbeeldingen vereist
            with output:
                print(f"❌ Onvoldoende foto's voor {person}, overslaan.")
            continue
        
        label_dict[idx] = person  # mapping van index naar persoonsnaam
        person_images[idx] = image_files

    if not person_images:
        with output:
            print("❌ Geen geldige afbeeldingen gevonden voor training.")
        return

    # Zorg dat alle labels evenveel afbeeldingen hebben
    min_images = min(len(imgs) for imgs in person_images.values())
    balanced_images = []
    balanced_labels = []
    
    for label, image_files in person_images.items():
        selected_files = np.random.choice(image_files, min_images, replace=False)
        for img_path in selected_files:
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_resized = cv2.resize(img, (100, 100))
            balanced_images.append(img_resized)
            balanced_labels.append(label)

    # Converteer naar numpy arrays en normaliseer
    images = np.array(balanced_images, dtype="float32") / 255.0
    labels = np.array(balanced_labels, dtype="int32")

    # Data Augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    augmented_images = []
    augmented_labels = []
    for img, label in zip(images, labels):
        img_expanded = np.expand_dims(img, axis=0)
        augment_iter = datagen.flow(img_expanded, batch_size=1)
        for _ in range(3):  # 3 augmented versies per afbeelding
            augmented_img = next(augment_iter)[0]
            augmented_images.append(augmented_img)
            augmented_labels.append(label)

    images = np.concatenate([images, np.array(augmented_images)], axis=0)
    labels = np.concatenate([labels, np.array(augmented_labels)], axis=0)

    # Shuffle de dataset
    images, labels = shuffle(images, labels, random_state=42)

    # Bouw het model met dropout ter voorkoming van overfitting
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(label_dict), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    model.fit(images, labels, epochs=15, batch_size=32, validation_split=0.2)

    # Sla het model en de labelmapping op
    model.save(MODEL_PATH)
    np.save(LABEL_PATH, label_dict)

    with output:
        print(f"✅ Model opgeslagen als {MODEL_PATH} en labels als {LABEL_PATH}")
        unique, counts = np.unique(labels, return_counts=True)
        print("Aantal afbeeldingen per label:", dict(zip(unique, counts)))
        print("Label mapping:", label_dict)

train_button.on_click(train_model)